In [9]:
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
#import seaborn as sns
from sklearn.metrics import f1_score,classification_report, log_loss
import warnings
warnings.filterwarnings("ignore")
pd.options.display.max_rows = 1000
from catboost import CatBoostRegressor, CatBoostClassifier
# np.random.seed(0)

from sklearn.metrics import mean_squared_error
#from sklearn.preprocessing import MinMaxScaler
#from xgboost import XGBClassifier

In [10]:
train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')


In [11]:
train.head(8)

,Applicant_ID,form_field1,form_field2,form_field3,form_field4,form_field5,form_field6,form_field7,form_field8,form_field9,...,form_field42,form_field43,form_field44,form_field45,form_field46,form_field47,form_field48,form_field49,form_field50,default_status
0,Apcnt_1000000,3436.0,0.28505,1.6560,0.0,0.0000,0.0,10689720.0,252072.0,4272776.0,...,0.392854,2.02,0.711632,0.0,0.0,charge,NaN,1.129518,0.044335,no
1,Apcnt_1000004,3456.0,0.67400,0.2342,0.0,0.0000,0.0,898979.0,497531.0,9073814.0,...,0.314281,8.08,0.183584,NaN,0.0,charge,349.805730,1.620483,0.322436,no
2,Apcnt_1000008,3276.0,0.53845,3.1510,0.0,6.2820,NaN,956940.0,NaN,192944.0,...,0.162965,18.18,0.791136,0.0,0.0,charge,NaN,1.513370,0.011640,yes
3,Apcnt_1000012,3372.0,0.17005,0.5050,0.0,0.0000,192166.0,3044703.0,385499.0,3986472.0,...,0.488884,2.02,0.685168,NaN,0.0,charge,89.940100,0.664452,0.082729,no
4,Apcnt_1000016,3370.0,0.77270,1.1010,0.0,0.0000,1556.0,214728.0,214728.0,1284089.0,...,0.275000,12.12,0.438168,0.0,0.0,charge,97.887502,1.427891,0.045630,no
5,Apcnt_1000020,3724.0,NaN,0.0000,0.0,0.0000,192944.0,9244585.0,6761209.0,67963357.0,...,0.030558,12.12,0.382960,NaN,0.0,lending,NaN,0.000000,NaN,no
6,Apcnt_1000024,3350.0,0.24205,0.3132,0.0,2.8188,NaN,580388.0,NaN,96472.0,...,0.660000,1.01,0.513504,NaN,0.0,charge,NaN,2.146512,0.004390,yes
7,Apcnt_1000028,3416.0,0.87020,0.0570,0.0,0.0000,17505.0,186331.0,186331.0,1484813.0,...,0.137500,3.03,0.635960,NaN,0.0,charge,155.295994,0.946074,0.128389,no


In [12]:
train.isna().sum()

Applicant_ID          0
form_field1        2529
form_field2        3844
form_field3         355
form_field4         355
form_field5         355
form_field6       13360
form_field7        5163
form_field8       13360
form_field9        8008
form_field10        355
form_field11      31421
form_field12       9895
form_field13       5889
form_field14          0
form_field15      22475
form_field16      13036
form_field17      11151
form_field18      10402
form_field19          4
form_field20        355
form_field21      15854
form_field22      20400
form_field23      28123
form_field24      13297
form_field25       5450
form_field26       7438
form_field27       9299
form_field28        355
form_field29        355
form_field30      25509
form_field31      39408
form_field32       5450
form_field33       1256
form_field34        355
form_field35      23148
form_field36       1995
form_field37       5450
form_field38        355
form_field39       4211
form_field40      43729
form_field41    

In [13]:
train.shape

(56000, 52)

In [14]:
train.drop(columns=['form_field48','form_field49'], inplace=True)
test.drop(columns=['form_field48','form_field49'], inplace=True)


In [15]:
test.shape

(24000, 49)

In [16]:
train.head()

,Applicant_ID,form_field1,form_field2,form_field3,form_field4,form_field5,form_field6,form_field7,form_field8,form_field9,...,form_field40,form_field41,form_field42,form_field43,form_field44,form_field45,form_field46,form_field47,form_field50,default_status
0,Apcnt_1000000,3436.0,0.28505,1.6560,0.0,0.000,0.0,10689720.0,252072.0,4272776.0,...,NaN,NaN,0.392854,2.02,0.711632,0.0,0.0,charge,0.044335,no
1,Apcnt_1000004,3456.0,0.67400,0.2342,0.0,0.000,0.0,898979.0,497531.0,9073814.0,...,NaN,NaN,0.314281,8.08,0.183584,NaN,0.0,charge,0.322436,no
2,Apcnt_1000008,3276.0,0.53845,3.1510,0.0,6.282,NaN,956940.0,NaN,192944.0,...,149.6805,NaN,0.162965,18.18,0.791136,0.0,0.0,charge,0.011640,yes
3,Apcnt_1000012,3372.0,0.17005,0.5050,0.0,0.000,192166.0,3044703.0,385499.0,3986472.0,...,NaN,NaN,0.488884,2.02,0.685168,NaN,0.0,charge,0.082729,no
4,Apcnt_1000016,3370.0,0.77270,1.1010,0.0,0.000,1556.0,214728.0,214728.0,1284089.0,...,150.0135,NaN,0.275000,12.12,0.438168,0.0,0.0,charge,0.045630,no


In [17]:
train['form_field8'].value_counts()

0.0           1772
192944.0       663
770998.0       526
1927495.0      452
385499.0       438
115533.0       400
1156497.0      397
5782485.0      337
3854990.0      329
578443.0       308
1541996.0      289
7709980.0      283
2312994.0      261
963942.0       235
9637475.0      200
3083992.0      185
1349441.0      168
2891437.0      158
1734940.0      157
11564970.0     152
289027.0       149
2698493.0      136
308477.0       136
4625988.0      136
231455.0       122
269966.0       117
154044.0       111
462521.0        99
77022.0         96
3469491.0       95
2120439.0       93
2505938.0       80
4240489.0       77
4818932.0       74
693976.0        71
5396986.0       70
6167984.0       67
5011487.0       66
6938982.0       62
848020.0        60
5204431.0       57
6553483.0       53
4433433.0       53
501032.0        53
3276936.0       52
5589930.0       52
4047934.0       51
3662435.0       50
11670.0         50
389.0           48
616954.0        48
96472.0         47
1272030.0   

In [18]:
#sns.heatmap(train.corr(), annot=True)

In [19]:
len_train = len(train)
df = pd.concat([train, test])

In [20]:
df['card_with_limit']=df['form_field19']-df['form_field17']



In [21]:
df['form_10_7']=df['form_field10']-df['form_field7']
df['form_8_6']=df['form_field8']-df['form_field6']
df['form_9_10']=df['form_field9']-df['form_field10']


In [22]:
df['default_status']=[0 if x=='no' else 1 for x in df['default_status']]
df['form_field47']=[1 if x=='charge' else 0 for x in df['form_field47']]

In [23]:
df.head()

,Applicant_ID,default_status,form_field1,form_field10,form_field11,form_field12,form_field13,form_field14,form_field15,form_field16,...,form_field5,form_field50,form_field6,form_field7,form_field8,form_field9,card_with_limit,form_10_7,form_8_6,form_9_10
0,Apcnt_1000000,0,3436.0,11333126.0,4397256.0,2301324.0,7875694.0,96375139,112181765.0,1.0,...,0.000,0.044335,0.0,10689720.0,252072.0,4272776.0,2.0,643406.0,252072.0,-7060350.0
1,Apcnt_1000004,0,3456.0,2533168.0,244292.0,3982971.0,2141445.0,28141427,45169902.0,1.0,...,0.000,0.322436,0.0,898979.0,497531.0,9073814.0,1.0,1634189.0,497531.0,6540646.0
2,Apcnt_1000008,1,3276.0,1079864.0,NaN,NaN,0.0,16576457,22452691.0,NaN,...,6.282,0.011640,NaN,956940.0,NaN,192944.0,NaN,122924.0,NaN,-886920.0
3,Apcnt_1000012,0,3372.0,3621979.0,NaN,771776.0,1186839.0,48187375,136297820.0,0.0,...,0.000,0.082729,192166.0,3044703.0,385499.0,3986472.0,1.0,577276.0,193333.0,364493.0
4,Apcnt_1000016,0,3370.0,361770.0,396391.0,818456.0,511535.0,28141427,15292757.0,0.0,...,0.000,0.045630,1556.0,214728.0,214728.0,1284089.0,3.0,147042.0,213172.0,922319.0


In [24]:
train = df[:len_train]
test = df[len_train:]

In [25]:
X = train.drop(columns=['Applicant_ID','default_status'])
y = train['default_status']
_id = test['Applicant_ID']
tes = test.drop(columns=['Applicant_ID','default_status'])

In [26]:
y.value_counts()

0    42285
1    13715
Name: default_status, dtype: int64

In [27]:
X.shape, tes.shape

((56000, 52), (24000, 52))

In [28]:
X = X.fillna(-99999)
tes = tes.fillna(-99999)

In [29]:
categorical_features_indices = np.where(X.dtypes != np.float)[0]; categorical_features_indices

array([ 5, 41], dtype=int64)

In [30]:
X.head()

,form_field1,form_field10,form_field11,form_field12,form_field13,form_field14,form_field15,form_field16,form_field17,form_field18,...,form_field5,form_field50,form_field6,form_field7,form_field8,form_field9,card_with_limit,form_10_7,form_8_6,form_9_10
0,3436.0,11333126.0,4397256.0,2301324.0,7875694.0,96375139,112181765.0,1.0,1.0,1.0,...,0.000,0.044335,0.0,10689720.0,252072.0,4272776.0,2.0,643406.0,252072.0,-7060350.0
1,3456.0,2533168.0,244292.0,3982971.0,2141445.0,28141427,45169902.0,1.0,1.0,1.0,...,0.000,0.322436,0.0,898979.0,497531.0,9073814.0,1.0,1634189.0,497531.0,6540646.0
2,3276.0,1079864.0,-99999.0,-99999.0,0.0,16576457,22452691.0,-99999.0,-99999.0,-99999.0,...,6.282,0.011640,-99999.0,956940.0,-99999.0,192944.0,-99999.0,122924.0,-99999.0,-886920.0
3,3372.0,3621979.0,-99999.0,771776.0,1186839.0,48187375,136297820.0,0.0,0.0,0.0,...,0.000,0.082729,192166.0,3044703.0,385499.0,3986472.0,1.0,577276.0,193333.0,364493.0
4,3370.0,361770.0,396391.0,818456.0,511535.0,28141427,15292757.0,0.0,0.0,0.0,...,0.000,0.045630,1556.0,214728.0,214728.0,1284089.0,3.0,147042.0,213172.0,922319.0


In [31]:
score=[]
test_pred=[]
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold,KFold
from sklearn.metrics import roc_auc_score
fold=StratifiedKFold(n_splits=10, shuffle=True, random_state =12345)

for train_index, test_index in fold.split(X,y):
    X_train, X_test = X.loc[train_index], X.loc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    cat=CatBoostClassifier(iterations=1500,logging_level='Silent', eval_metric='AUC')
    cat.fit(X_train,y_train,eval_set=[(X_train,y_train),(X_test, y_test)], 
            early_stopping_rounds=100,verbose=True)
    
    preds=cat.predict_proba(X_test)[:,1]
    print("ROC AUC Score: ",roc_auc_score(y_test,preds))
    
    score.append(roc_auc_score(y_test,preds))
    p2 = cat.predict_proba(tes)[:,1]
    test_pred.append(p2)

Learning rate set to 0.099001
0:	test: 0.7897108	test1: 0.7842253	best: 0.7842253 (0)	total: 171ms	remaining: 4m 16s
1:	test: 0.8088312	test1: 0.8043481	best: 0.8043481 (1)	total: 290ms	remaining: 3m 37s
2:	test: 0.8142711	test1: 0.8111675	best: 0.8111675 (2)	total: 427ms	remaining: 3m 33s
3:	test: 0.8167348	test1: 0.8109015	best: 0.8111675 (2)	total: 547ms	remaining: 3m 24s
4:	test: 0.8201481	test1: 0.8144460	best: 0.8144460 (4)	total: 657ms	remaining: 3m 16s
5:	test: 0.8214348	test1: 0.8151568	best: 0.8151568 (5)	total: 724ms	remaining: 3m
6:	test: 0.8225521	test1: 0.8162865	best: 0.8162865 (6)	total: 787ms	remaining: 2m 47s
7:	test: 0.8241902	test1: 0.8177031	best: 0.8177031 (7)	total: 895ms	remaining: 2m 46s
8:	test: 0.8256240	test1: 0.8189144	best: 0.8189144 (8)	total: 1s	remaining: 2m 46s
9:	test: 0.8265161	test1: 0.8193305	best: 0.8193305 (9)	total: 1.08s	remaining: 2m 41s
10:	test: 0.8271789	test1: 0.8197873	best: 0.8197873 (10)	total: 1.16s	remaining: 2m 37s
11:	test: 0.827649

95:	test: 0.8504622	test1: 0.8353905	best: 0.8353905 (95)	total: 8.04s	remaining: 1m 57s
96:	test: 0.8505626	test1: 0.8353799	best: 0.8353905 (95)	total: 8.11s	remaining: 1m 57s
97:	test: 0.8506935	test1: 0.8353866	best: 0.8353905 (95)	total: 8.17s	remaining: 1m 56s
98:	test: 0.8509094	test1: 0.8354090	best: 0.8354090 (98)	total: 8.23s	remaining: 1m 56s
99:	test: 0.8510307	test1: 0.8354781	best: 0.8354781 (99)	total: 8.31s	remaining: 1m 56s
100:	test: 0.8512082	test1: 0.8354405	best: 0.8354781 (99)	total: 8.37s	remaining: 1m 55s
101:	test: 0.8513330	test1: 0.8353392	best: 0.8354781 (99)	total: 8.44s	remaining: 1m 55s
102:	test: 0.8515375	test1: 0.8353697	best: 0.8354781 (99)	total: 8.54s	remaining: 1m 55s
103:	test: 0.8517779	test1: 0.8354931	best: 0.8354931 (103)	total: 8.63s	remaining: 1m 55s
104:	test: 0.8520066	test1: 0.8357263	best: 0.8357263 (104)	total: 8.69s	remaining: 1m 55s
105:	test: 0.8521044	test1: 0.8357654	best: 0.8357654 (105)	total: 8.76s	remaining: 1m 55s
106:	test: 0

187:	test: 0.8664876	test1: 0.8376821	best: 0.8380006 (175)	total: 14.6s	remaining: 1m 41s
188:	test: 0.8666426	test1: 0.8376280	best: 0.8380006 (175)	total: 14.7s	remaining: 1m 41s
189:	test: 0.8667869	test1: 0.8375720	best: 0.8380006 (175)	total: 14.7s	remaining: 1m 41s
190:	test: 0.8669658	test1: 0.8375606	best: 0.8380006 (175)	total: 14.8s	remaining: 1m 41s
191:	test: 0.8670927	test1: 0.8375844	best: 0.8380006 (175)	total: 14.9s	remaining: 1m 41s
192:	test: 0.8672490	test1: 0.8374598	best: 0.8380006 (175)	total: 14.9s	remaining: 1m 41s
193:	test: 0.8673745	test1: 0.8374410	best: 0.8380006 (175)	total: 15s	remaining: 1m 40s
194:	test: 0.8675820	test1: 0.8374525	best: 0.8380006 (175)	total: 15.1s	remaining: 1m 40s
195:	test: 0.8677440	test1: 0.8374773	best: 0.8380006 (175)	total: 15.1s	remaining: 1m 40s
196:	test: 0.8678925	test1: 0.8375623	best: 0.8380006 (175)	total: 15.2s	remaining: 1m 40s
197:	test: 0.8680849	test1: 0.8375559	best: 0.8380006 (175)	total: 15.3s	remaining: 1m 40s
1

281:	test: 0.8794882	test1: 0.8380640	best: 0.8380640 (281)	total: 21.2s	remaining: 1m 31s
282:	test: 0.8796533	test1: 0.8380235	best: 0.8380640 (281)	total: 21.2s	remaining: 1m 31s
283:	test: 0.8797991	test1: 0.8379629	best: 0.8380640 (281)	total: 21.3s	remaining: 1m 31s
284:	test: 0.8799279	test1: 0.8379068	best: 0.8380640 (281)	total: 21.3s	remaining: 1m 31s
285:	test: 0.8801143	test1: 0.8379537	best: 0.8380640 (281)	total: 21.4s	remaining: 1m 30s
286:	test: 0.8802736	test1: 0.8379856	best: 0.8380640 (281)	total: 21.5s	remaining: 1m 30s
287:	test: 0.8804304	test1: 0.8379618	best: 0.8380640 (281)	total: 21.6s	remaining: 1m 30s
288:	test: 0.8804344	test1: 0.8379432	best: 0.8380640 (281)	total: 21.6s	remaining: 1m 30s
289:	test: 0.8805913	test1: 0.8379492	best: 0.8380640 (281)	total: 21.7s	remaining: 1m 30s
290:	test: 0.8806574	test1: 0.8379660	best: 0.8380640 (281)	total: 21.8s	remaining: 1m 30s
291:	test: 0.8807718	test1: 0.8378896	best: 0.8380640 (281)	total: 21.8s	remaining: 1m 30s

375:	test: 0.8905102	test1: 0.8379125	best: 0.8380640 (281)	total: 27.9s	remaining: 1m 23s
376:	test: 0.8906500	test1: 0.8379273	best: 0.8380640 (281)	total: 28s	remaining: 1m 23s
377:	test: 0.8907044	test1: 0.8378953	best: 0.8380640 (281)	total: 28s	remaining: 1m 23s
378:	test: 0.8907554	test1: 0.8379068	best: 0.8380640 (281)	total: 28.1s	remaining: 1m 23s
379:	test: 0.8907731	test1: 0.8379186	best: 0.8380640 (281)	total: 28.2s	remaining: 1m 23s
380:	test: 0.8908572	test1: 0.8379404	best: 0.8380640 (281)	total: 28.2s	remaining: 1m 22s
381:	test: 0.8910357	test1: 0.8379980	best: 0.8380640 (281)	total: 28.3s	remaining: 1m 22s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.83806402
bestIteration = 281

Shrink model to first 282 iterations.
ROC AUC Score:  0.8380640199869429
Learning rate set to 0.099001
0:	test: 0.7893542	test1: 0.7871014	best: 0.7871014 (0)	total: 173ms	remaining: 4m 18s
1:	test: 0.8052568	test1: 0.8042318	best: 0.8042318 (1)	total: 244ms	remaining

...

277:	test: 0.8776176	test1: 0.8419908	best: 0.8422933 (259)	total: 19.6s	remaining: 1m 26s
278:	test: 0.8777981	test1: 0.8420164	best: 0.8422933 (259)	total: 19.7s	remaining: 1m 26s
279:	test: 0.8779241	test1: 0.8420716	best: 0.8422933 (259)	total: 19.8s	remaining: 1m 26s
280:	test: 0.8780621	test1: 0.8420326	best: 0.8422933 (259)	total: 19.8s	remaining: 1m 26s
281:	test: 0.8780686	test1: 0.8420326	best: 0.8422933 (259)	total: 19.9s	remaining: 1m 26s
282:	test: 0.8781580	test1: 0.8419701	best: 0.8422933 (259)	total: 20s	remaining: 1m 26s
283:	test: 0.8783054	test1: 0.8418977	best: 0.8422933 (259)	total: 20.1s	remaining: 1m 25s
284:	test: 0.8785078	test1: 0.8419282	best: 0.8422933 (259)	total: 20.1s	remaining: 1m 25s
285:	test: 0.8786886	test1: 0.8419261	best: 0.8422933 (259)	total: 20.2s	remaining: 1m 25s
286:	test: 0.8788625	test1: 0.8419387	best: 0.8422933 (259)	total: 20.3s	remaining: 1m 25s
287:	test: 0.8790138	test1: 0.8418897	best: 0.8422933 (259)	total: 20.3s	remaining: 1m 25s
2

7:	test: 0.8256233	test1: 0.8231819	best: 0.8231819 (7)	total: 703ms	remaining: 2m 11s
8:	test: 0.8264837	test1: 0.8236623	best: 0.8236623 (8)	total: 773ms	remaining: 2m 8s
9:	test: 0.8275888	test1: 0.8243532	best: 0.8243532 (9)	total: 840ms	remaining: 2m 5s
10:	test: 0.8289202	test1: 0.8251119	best: 0.8251119 (10)	total: 923ms	remaining: 2m 4s
11:	test: 0.8290228	test1: 0.8249605	best: 0.8251119 (10)	total: 989ms	remaining: 2m 2s
12:	test: 0.8293890	test1: 0.8245591	best: 0.8251119 (10)	total: 1.05s	remaining: 2m
13:	test: 0.8304850	test1: 0.8252973	best: 0.8252973 (13)	total: 1.12s	remaining: 1m 59s
14:	test: 0.8311227	test1: 0.8259663	best: 0.8259663 (14)	total: 1.19s	remaining: 1m 57s
15:	test: 0.8317824	test1: 0.8269455	best: 0.8269455 (15)	total: 1.26s	remaining: 1m 56s
16:	test: 0.8324058	test1: 0.8276626	best: 0.8276626 (16)	total: 1.34s	remaining: 1m 56s
17:	test: 0.8327404	test1: 0.8282447	best: 0.8282447 (17)	total: 1.42s	remaining: 1m 57s
18:	test: 0.8332659	test1: 0.828766

101:	test: 0.8513270	test1: 0.8366712	best: 0.8366925 (100)	total: 7.46s	remaining: 1m 42s
102:	test: 0.8514770	test1: 0.8366204	best: 0.8366925 (100)	total: 7.52s	remaining: 1m 42s
103:	test: 0.8516143	test1: 0.8365821	best: 0.8366925 (100)	total: 7.59s	remaining: 1m 41s
104:	test: 0.8517772	test1: 0.8366397	best: 0.8366925 (100)	total: 7.66s	remaining: 1m 41s
105:	test: 0.8518854	test1: 0.8365631	best: 0.8366925 (100)	total: 7.72s	remaining: 1m 41s
106:	test: 0.8519792	test1: 0.8365752	best: 0.8366925 (100)	total: 7.78s	remaining: 1m 41s
107:	test: 0.8520636	test1: 0.8365374	best: 0.8366925 (100)	total: 7.85s	remaining: 1m 41s
108:	test: 0.8522278	test1: 0.8364696	best: 0.8366925 (100)	total: 7.93s	remaining: 1m 41s
109:	test: 0.8523771	test1: 0.8364075	best: 0.8366925 (100)	total: 8.02s	remaining: 1m 41s
110:	test: 0.8525679	test1: 0.8364106	best: 0.8366925 (100)	total: 8.08s	remaining: 1m 41s
111:	test: 0.8527042	test1: 0.8364090	best: 0.8366925 (100)	total: 8.15s	remaining: 1m 41s

195:	test: 0.8664877	test1: 0.8374740	best: 0.8378974 (189)	total: 14s	remaining: 1m 33s
196:	test: 0.8666681	test1: 0.8374784	best: 0.8378974 (189)	total: 14.1s	remaining: 1m 33s
197:	test: 0.8668932	test1: 0.8373742	best: 0.8378974 (189)	total: 14.2s	remaining: 1m 33s
198:	test: 0.8670519	test1: 0.8374521	best: 0.8378974 (189)	total: 14.2s	remaining: 1m 33s
199:	test: 0.8672114	test1: 0.8373348	best: 0.8378974 (189)	total: 14.3s	remaining: 1m 33s
200:	test: 0.8673482	test1: 0.8373478	best: 0.8378974 (189)	total: 14.4s	remaining: 1m 32s
201:	test: 0.8674469	test1: 0.8373091	best: 0.8378974 (189)	total: 14.4s	remaining: 1m 32s
202:	test: 0.8676155	test1: 0.8373842	best: 0.8378974 (189)	total: 14.5s	remaining: 1m 32s
203:	test: 0.8677500	test1: 0.8373905	best: 0.8378974 (189)	total: 14.6s	remaining: 1m 32s
204:	test: 0.8678466	test1: 0.8373723	best: 0.8378974 (189)	total: 14.7s	remaining: 1m 32s
205:	test: 0.8680373	test1: 0.8372644	best: 0.8378974 (189)	total: 14.7s	remaining: 1m 32s
2

287:	test: 0.8791319	test1: 0.8370980	best: 0.8378974 (189)	total: 20.6s	remaining: 1m 26s
288:	test: 0.8792398	test1: 0.8370876	best: 0.8378974 (189)	total: 20.6s	remaining: 1m 26s
289:	test: 0.8793482	test1: 0.8370752	best: 0.8378974 (189)	total: 20.7s	remaining: 1m 26s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.8378973976
bestIteration = 189

Shrink model to first 190 iterations.
ROC AUC Score:  0.837897397572503


In [32]:
test_predx = np.mean(test_pred, axis =0)

In [33]:
fin = pd.DataFrame({'Applicant_ID':test.Applicant_ID, 'default_status':test_predx})
fin.to_csv('subder.csv', index=False)

In [34]:
a = pd.read_csv('subder.csv')
b= pd.read_csv('checking25.csv')

In [35]:
join_pred=(a['default_status']+b['default_status'])/2

In [36]:
final = pd.DataFrame({'Applicant_ID':test.Applicant_ID, 'default_status':join_pred})
final.to_csv('subder60.csv', index=False)